In [4]:
import sys 
!{sys.executable} -m pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import when

spark = SparkSession.builder.appName("CustomerOrdersJob").getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/28 14:35:36 WARN Utils: Your hostname, DIVAs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
25/12/28 14:35:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/28 14:35:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
customers_path = "/Users/divamodi/pyspark-project/spark-apps/test_input/customers.csv"
orders_path = "/Users/divamodi/pyspark-project/spark-apps/test_input/orders.json"
output_path_csv = "/Users/divamodi/pyspark-project/spark-apps/output/tmp/orders_enriched_csv"
output_path_parquet = "/Users/divamodi/pyspark-project/spark-apps/output/tmp/orders_enriched_parquet"

In [6]:
df_customers = spark.read.option("header", True).csv(customers_path)

In [22]:
df_orders = spark.read.json(orders_path)

In [23]:
df_orders.show()

+------+-----------+--------+----------+
|amount|customer_id|order_id|    status|
+------+-----------+--------+----------+
| 250.5|        101|    5001|   shipped|
| 145.0|        102|    5002| cancelled|
|389.99|        104|    5003|processing|
| 89.99|        105|    5004|   shipped|
| 199.0|        106|    5005|  returned|
+------+-----------+--------+----------+



In [10]:
import os, pathlib
print("cwd:", os.getcwd())
print("notebook dir guess:", pathlib.Path().resolve())

cwd: /Users/divamodi/pyspark-project/notebooks/pyspark/practice
notebook dir guess: /Users/divamodi/pyspark-project/notebooks/pyspark/practice


In [24]:
df_joined = df_orders.join(df_customers, on="customer_id", how="inner")

df_enriched = df_joined.withColumn(
    "order_type",
    when(df_joined.amount >= 200, "High Value")
    .when(df_joined.amount >= 100, "Medium Value")
    .otherwise("Low Value")
)

df_enriched.select("order_id", "name", "amount", "order_type").show()

df_enriched_op = df_enriched.select("order_id", "name", "amount", "order_type")
df_enriched_op.write.mode("overwrite").option("header", True).csv(output_path_csv)

+--------+----------+------+------------+
|order_id|      name|amount|  order_type|
+--------+----------+------+------------+
|    5001|  John Doe| 250.5|  High Value|
|    5002|Jane Smith| 145.0|Medium Value|
|    5003|Rita Mehra|389.99|  High Value|
|    5004| Wei Zhang| 89.99|   Low Value|
+--------+----------+------+------------+



In [25]:
output_path_csv


'/Users/divamodi/pyspark-project/spark-apps/output/tmp/orders_enriched_csv'

In [26]:
df_enriched_op.write.mode("overwrite").parquet(output_path_parquet)

In [7]:
df_customers.createOrReplaceTempView('customers')

In [9]:
spark.sql("""
    select * from customers
""").show()

+-----------+----------+----------------+---+-------+
|customer_id|      name|           email|age|country|
+-----------+----------+----------------+---+-------+
|        101|  John Doe|john@example.com| 28|     US|
|        102|Jane Smith|jane@example.com| 34|     UK|
|        103|  Ali Khan| ali@example.com| 40|    UAE|
|        104|Rita Mehra|rita@example.com| 25|  India|
|        105| Wei Zhang| wei@example.com| 31|  China|
+-----------+----------+----------------+---+-------+

